In [ ]:
# Exercise: Titanic Dataset - Types of Data and Handling Missing Data

Briefly explain types of data and why we have to handle missing values.

## Preparing data

Reload Titaic


In [173]:
import pandas as pd

# Load data from our dataset file into a pandas dataframe
dataset = pd.read_csv('Data/titanic.csv', index_col=False, sep=",",header=0)

# Let's take a look at the data
dataset.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Discuss different types of data above; show code to display data type (ex: dataset.info()).





## Handling Missing Data

- Recall what data is missing
- Show 3 different ways to deal with missing data (one for each column)
    - Embarked: Delete data
    - Cabin: create new category
    - Age: Use mean
- Save cleaned data to a "clean" dataset

In [175]:
# Calculate the number of empty cells in each column
# and store it in a new dataframe
missing_data = dataset.isnull().sum().to_frame()

# Rename column holding the sums
missing_data = missing_data.rename(columns={0:'Empty Cells'})

# Print the results
print(missing_data)

             Empty Cells
PassengerId            0
Survived               0
Pclass                 0
Name                   0
Sex                    0
Age                  177
SibSp                  0
Parch                  0
Ticket                 0
Fare                   0
Cabin                687
Embarked               2


**Before** starting  - remove categorical data so we can still use LinReg without one hot encoding.

## Building a Model with Cleaned Data

- Split model
- Train
- Calculate loss



## Building a Model with Uncleaned Data

Split model
Train
Calculate loss

Compare losses

## Summary
.....
Recall types of data, transformation to handle missing data and draw concclusion on model comparison.
